**Install dependencies**

In [ ]:
!pip install -q flask pyngrok yfinance google-generativeai faiss-cpu

**Gemini API Key set up**

Step-1: Go to Google AI Studio https://ai.google.dev/?utm_source=chatgpt.com

Step-2: Sign in with your Google account.

Step-3: Open the “API Keys” page. In the left sidebar, click on “API keys”.

Step-4: Create or select a project.

Step-5: Create a new API key.

Step-6: Copy and store the key safely.

**Create app.py**

In [ ]:
%%writefile app.py
# =====================================================
# 💹 Finance Agent — Flask Web Deployment
# FAISS Memory + Gemini + Yahoo Finance
# =====================================================

# 1. IMPORTS & GLOBAL CONFIG
# -----------------------------------------------------
import os
import re
import datetime as dt
from textwrap import dedent

import numpy as np
import pandas as pd
import yfinance as yf
import faiss
import google.generativeai as genai

from flask import Flask, render_template, request

# -----------------------------------------------------
# 2. GEMINI SETUP (LLM + EMBEDDINGS)
# -----------------------------------------------------

# Include the Gemini API key
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "INPUT_YOUR_GEMINI_API_KEY_HERE")

genai.configure(api_key=GEMINI_API_KEY)

# Model names (same as your original code)
GENERATION_MODEL_NAME = "gemini-2.5-flash"
EMBEDDING_MODEL_NAME = "models/text-embedding-004"

generation_model = genai.GenerativeModel(GENERATION_MODEL_NAME)


def call_gemini(prompt: str) -> str:
    """Call Gemini text model with a single prompt and return the response text."""
    resp = generation_model.generate_content(prompt)
    return resp.text.strip() if getattr(resp, "text", None) else ""


def embed_text(text: str) -> np.ndarray:
    """Get an embedding vector for the given text using Gemini embeddings."""
    resp = genai.embed_content(
        model=EMBEDDING_MODEL_NAME,
        content=text,
        task_type="retrieval_document",
    )
    vec = np.array(resp["embedding"], dtype="float32")
    return vec


# -----------------------------------------------------
# 3. FAISS-BASED MEMORY STORE
# -----------------------------------------------------
class FaissMemory:
    """
    Simple in-memory FAISS index for semantic memory.
    Stores (embedding, text, metadata) and supports similarity search.
    """

    def __init__(self):
        self.index = None          # faiss.IndexFlatIP
        self.texts = []            # list of strings
        self.metadatas = []        # list of dicts
        self._id_counter = 0

    def _ensure_index(self, dim: int):
        """Create the FAISS index lazily with given dimension."""
        if self.index is None:
            self.index = faiss.IndexFlatIP(dim)

    def add(self, text: str, metadata: dict = None):
        """Add a text + metadata entry to the memory."""
        if not text.strip():
            return
        vec = embed_text(text)
        self._ensure_index(len(vec))
        vec = vec.reshape(1, -1)
        self.index.add(vec)
        self.texts.append(text)
        self.metadatas.append(metadata or {})
        self._id_counter += 1

    def search(self, query: str, k: int = 5):
        """Return top-k (text, metadata, score) for a query text."""
        if self.index is None or len(self.texts) == 0:
            return []
        q_vec = embed_text(query).reshape(1, -1)
        scores, idxs = self.index.search(q_vec, min(k, len(self.texts)))

        results = []
        for score, idx in zip(scores[0], idxs[0]):
            if idx < 0:
                continue
            results.append({
                "text": self.texts[idx],
                "metadata": self.metadatas[idx],
                "score": float(score),
            })
        return results


# Single FAISS memory instance for the whole app
memory = FaissMemory()

# Structured symbol-specific memory (Option A)
latest_equity_analysis = {}     # symbol -> last full equity report (string)
latest_crypto_analysis = {}     # symbol -> last full crypto report (string)
latest_portfolio_analysis = None

# Last context for follow-ups
last_context = {
    "type": None,   # "stock", "crypto", "portfolio", or None
    "symbol": None  # e.g., "AAPL" or "BTC-USD"
}


# -----------------------------------------------------
# 4. FINANCE HELPERS (UNCHANGED LOGIC)
# -----------------------------------------------------
def compute_max_drawdown(series: pd.Series) -> float:
    """
    Compute max drawdown from a price or portfolio value series.
    Result is between 0 and -1 (0% to -100%).
    """
    if series is None or series.empty:
        return float(0.0)

    # If this looks like returns, convert to cumulative equity curve
    if series.index.is_monotonic_increasing and (series <= 1.5).all() and (series >= -1.0).all():
        cumulative = (1 + series.fillna(0)).cumprod()
    else:
        cumulative = series.astype(float)

    peak = cumulative.cummax()
    drawdown = (cumulative - peak) / peak
    return float(drawdown.min())


def _build_latest_snapshot_from_history_and_info(history: pd.DataFrame, info: dict, last_date: str):
    """
    Helper to construct a 'latest_snapshot' dict from yfinance history + info.
    Used for both equities and crypto.
    """
    latest_snapshot = {}

    if history is not None and not history.empty:
        last_row = history.iloc[-1]

        def _safe_get(row, key):
            if key in row:
                try:
                    return float(row[key])
                except Exception:
                    return None
            return None

        latest_snapshot.update({
            "as_of": last_date,
            "close": _safe_get(last_row, "Close"),
            "open": _safe_get(last_row, "Open"),
            "high": _safe_get(last_row, "High"),
            "low": _safe_get(last_row, "Low"),
            "volume": _safe_get(last_row, "Volume"),
        })

        # 1-year high/low from historical close prices
        try:
            latest_snapshot["one_year_high"] = float(history["Close"].max())
            latest_snapshot["one_year_low"] = float(history["Close"].min())
        except Exception:
            pass

    def _safe_float(x):
        try:
            return float(x)
        except Exception:
            return None

    if info:
        latest_snapshot.update({
            "real_time_price": _safe_float(info.get("currentPrice") or info.get("regularMarketPrice")),
            "real_time_change": _safe_float(info.get("regularMarketChange")),
            "real_time_change_pct": _safe_float(info.get("regularMarketChangePercent")),
            "market_cap": info.get("marketCap"),
            "fifty_two_week_high": _safe_float(info.get("fiftyTwoWeekHigh")),
            "fifty_two_week_low": _safe_float(info.get("fiftyTwoWeekLow")),
        })

    return latest_snapshot


def get_equity_data(symbol: str) -> dict:
    """Fetch 1-year equity data and metadata from Yahoo Finance."""
    symbol = symbol.upper()
    ticker = yf.Ticker(symbol)

    try:
        history = ticker.history(period="1y")
    except Exception:
        history = pd.DataFrame()

    if history is None:
        history = pd.DataFrame()

    try:
        info = ticker.info
    except Exception:
        info = {}

    try:
        news = ticker.news
    except Exception:
        news = []

    first_date = history.index.min().strftime("%Y-%m-%d") if not history.empty else None
    last_date = history.index.max().strftime("%Y-%m-%d") if not history.empty else None

    latest_snapshot = _build_latest_snapshot_from_history_and_info(history, info, last_date)

    data = {
        "symbol": symbol,
        "info": info,
        "history": history,
        "news": news,
        "first_date": first_date,
        "last_date": last_date,
        "latest_snapshot": latest_snapshot,
    }
    return data


def get_crypto_data(symbol: str) -> dict:
    """Fetch 1-year crypto data (e.g., BTC-USD) from Yahoo Finance."""
    symbol = symbol.upper()
    ticker = yf.Ticker(symbol)

    try:
        history = ticker.history(period="1y")
    except Exception:
        history = pd.DataFrame()

    if history is None:
        history = pd.DataFrame()

    try:
        info = ticker.info
    except Exception:
        info = {}

    try:
        news = ticker.news
    except Exception:
        news = []

    first_date = history.index.min().strftime("%Y-%m-%d") if not history.empty else None
    last_date = history.index.max().strftime("%Y-%m-%d") if not history.empty else None

    latest_snapshot = _build_latest_snapshot_from_history_and_info(history, info, last_date)

    data = {
        "symbol": symbol,
        "info": info,
        "history": history,
        "news": news,
        "first_date": first_date,
        "last_date": last_date,
        "latest_snapshot": latest_snapshot,
    }
    return data


def get_portfolio_history(holdings, period: str = "1y") -> dict:
    """
    holdings: list of dicts like [{"symbol": "AAPL", "weight": 0.4}, ...]
    """
    symbols = [h["symbol"].upper() for h in holdings]
    weights = np.array([h["weight"] for h in holdings], dtype=float)

    # Normalize weights if they don’t sum to 1
    if weights.sum() <= 0:
        raise ValueError("Total weights must be > 0.")
    if not np.isclose(weights.sum(), 1.0):
        weights = weights / weights.sum()

    try:
        raw = yf.download(symbols, period=period)
    except Exception as e:
        raise ValueError(f"Error downloading data for portfolio: {e}")

    if "Close" not in raw.columns and not isinstance(raw, pd.Series):
        raise ValueError("Downloaded data has no 'Close' prices.")

    data = raw["Close"] if "Close" in raw.columns else raw

    if isinstance(data, pd.Series):
        data = data.to_frame()

    data = data.dropna()
    if data.empty:
        raise ValueError("No price history available for the given symbols/period.")

    # Compute daily returns
    returns = data.pct_change().dropna()

    # Portfolio daily returns
    portfolio_returns = returns.dot(weights)

    # Annualized return and volatility (assuming 252 trading days)
    ann_return = (1 + portfolio_returns.mean())**252 - 1
    ann_vol = portfolio_returns.std() * np.sqrt(252)
    sharpe = ann_return / ann_vol if ann_vol > 0 else 0.0

    # Max drawdown (on cumulative portfolio value)
    equity_curve = (1 + portfolio_returns).cumprod()
    max_dd = compute_max_drawdown(equity_curve)

    # Per-asset volatility contribution (simple approximation)
    asset_vol = returns.std() * np.sqrt(252)
    vol_contrib = weights * asset_vol.values

    # Correlation matrix
    corr = returns.corr()

    first_date = data.index.min().strftime("%Y-%m-%d")
    last_date = data.index.max().strftime("%Y-%m-%d")

    return {
        "symbols": symbols,
        "weights": weights,
        "price_history": data,
        "returns": returns,
        "portfolio_returns": portfolio_returns,
        "annual_return": float(ann_return),
        "annual_volatility": float(ann_vol),
        "sharpe_ratio": float(sharpe),
        "vol_contrib": vol_contrib,
        "corr": corr,
        "max_drawdown": float(max_dd),
        "first_date": first_date,
        "last_date": last_date,
    }


# -----------------------------------------------------
# 5. PROMPT BUILDERS (UNCHANGED LOGIC)
# -----------------------------------------------------
def build_equity_prompt(symbol: str, data: dict) -> str:
    return dedent(f"""
        You are a senior equity research analyst.

        Below is real market and fundamental data for the stock {symbol} from Yahoo Finance.

        =======================
        DATA RANGE (IMPORTANT)
        =======================
        - First available date: {data.get("first_date", "N/A")}
        - Last available date:  {data.get("last_date", "N/A")}
        (All analysis must reference ONLY this date range. DO NOT invent other dates or years.)

        =======================
        LATEST SNAPSHOT (MOST RECENT DAY)
        =======================
        {data.get("latest_snapshot", {})}

        =======================
        RAW DATA (JSON-LIKE)
        =======================
        {data}

        =======================
        TASK
        =======================

        Write a structured equity research report for {symbol} with these sections:

        1. Executive Summary
           - 3–5 key bullets
           - Overall sentiment: Bullish / Bearish / Neutral

        2. Market Snapshot
           - Current price, day change, 52-week range, market cap

        3. Valuation & Key Metrics
           - P/E, Forward P/E, EPS, Revenue growth, Profit margin, ROE
           - Comment on whether valuation seems stretched or reasonable
           - Compare to typical market / tech sector multiples (approximate is ok)

        4. Business & Financial Health
           - Revenue and earnings trend (QoQ or YoY)
           - Balance sheet highlights (debt, cash, leverage)
           - Cash flow quality

        5. Analyst & News View
           - If available: analyst recommendations & price targets
           - Summarize recent news: classify as Bullish / Neutral / Bearish

        6. Risks
           - At least 3 concrete risks (competitive, macro, regulatory, execution)

        7. Forward-Looking View
           - Growth drivers
           - Where this stock fits in a portfolio
           - Who this stock may be suitable for (long-term, growth, defensive, etc.)

        End with this sentence exactly:
        "This analysis is for informational purposes only and is not financial advice."
    """)


def build_crypto_prompt(symbol: str, data: dict) -> str:
    return dedent(f"""
        You are a professional crypto market analyst.

        Below is market data and recent news for the crypto asset {symbol} taken directly from Yahoo Finance.

        =======================
        DATA RANGE (IMPORTANT)
        =======================
        - First available date: {data.get("first_date", "N/A")}
        - Last available date:  {data.get("last_date", "N/A")}
        (All analysis must only reference this timeframe. Do NOT assume any other dates.)

        =======================
        LATEST SNAPSHOT (MOST RECENT DAY)
        =======================
        {data.get("latest_snapshot", {})}

        =======================
        RAW DATA
        =======================
        {data}

        =======================
        IMPORTANT RULES (DO NOT BREAK)
        =======================
        - Use ONLY information present in the RAW DATA section.
        - If the RAW DATA does NOT explicitly mention an upgrade, catalyst, ETF, regulatory event,
          institution, or narrative, DO NOT invent one.
        - If the news mentions an upgrade, refer to it exactly as written.
        - If no upgrade is mentioned, write: "No protocol upgrade is referenced in the provided news."
        - Do NOT guess future price levels, upgrade names, institutions, or catalysts.
        - If information is missing, say "Not provided in the data."

        =======================
        TASK
        =======================

        Produce a structured crypto analysis for {symbol} using ONLY the information from RAW DATA:

        1. Overview
           - Identify if the asset is a coin or token (if identifiable from data)
           - Main purpose or narrative (only if supported by RAW DATA; otherwise say "Not specified in data")

        2. Market Snapshot
           - Current price, 24h change, market cap (if available)
           - 1-year high/low (from provided historical data)
           - Compare current price vs 50-day and 200-day averages (above/below)

        3. Trend & Volatility
           - Identify trend (uptrend / downtrend / sideways) based ONLY on historical data provided
           - Describe volatility level relative to typical crypto (high / moderate / low)

        4. Narrative & Catalysts
           - Summarize ONLY the themes and events actually present in the news
             (e.g., macro, regulation, institutional flows, ETF mentions)
           - If no catalysts are mentioned, clearly state:
             "No major catalysts mentioned in the provided news."
           - Give a simple sentiment label: Bullish / Neutral / Bearish (based on news tone)

        5. Risks
           - List at least 3 risks, using RAW DATA when available
             (e.g., volatility, regulatory, macro, competition)
           - Do NOT invent blockchain upgrades, forks, or technical developments.

        6. Investor Profile Fit
           - Briefly describe who might consider this asset (based on typical risk characteristics)

        End with this sentence exactly:
        "This is not investment advice or a recommendation to buy or sell any asset."
    """)


def build_portfolio_prompt(portfolio_stats: dict, holdings: list) -> str:
    return dedent(f"""
        You are a professional portfolio analyst.

        Below is the user's portfolio and performance statistics computed from 1-year daily data.

        =======================
        DATA RANGE (IMPORTANT)
        =======================
        - First available date: {portfolio_stats.get("first_date", "N/A")}
        - Last available date:  {portfolio_stats.get("last_date", "N/A")}
        (All analysis must reference ONLY this date range. Do NOT invent any other dates or years.)

        =======================
        HOLDINGS
        =======================
        {holdings}

        =======================
        PORTFOLIO STATS
        =======================
        - Symbols: {portfolio_stats['symbols']}
        - Weights: {portfolio_stats['weights'].tolist()}
        - Annualized Return: {portfolio_stats['annual_return']:.4f}
        - Annualized Volatility: {portfolio_stats['annual_volatility']:.4f}
        - Approx. Sharpe Ratio (rf=0): {portfolio_stats['sharpe_ratio']:.4f}
        - Max Drawdown: {portfolio_stats['max_drawdown']:.4f}
        - Per-asset volatility contribution: {portfolio_stats['vol_contrib'].tolist()}
        - Correlation Matrix: {portfolio_stats['corr'].round(3).values.tolist()}

        =======================
        TASK
        =======================

        Write a structured portfolio analysis report with the following sections:

        1. Overview
           - Describe what kind of portfolio this resembles (growth, tech-heavy, defensive, concentrated).
           - Comment on sector exposure (infer sectors from ticker names).

        2. Risk & Return Profile
           - Interpret annual return, volatility, Sharpe ratio, and max drawdown.
           - Clearly classify risk level (low / medium / high).

        3. Diversification & Concentration
           - Comment on number of holdings, sector exposure, and concentration risk.
           - Use the correlation matrix to explain whether the portfolio benefits from diversification.

        4. Contribution Analysis
           - Explain which assets likely contribute most to overall volatility and performance.
           - Refer to weights and per-asset volatility contribution.

        5. Suggestions (High-Level, Not Financial Advice)
           - Provide 3–5 principle-based suggestions
             (e.g., sector diversification, adding low-correlation assets, balancing weights).

        End the report with:
        "This portfolio analysis is informational only and not financial advice."
    """)


def build_chat_prompt(user_message: str, retrieved_docs: list) -> str:
    """
    Build a prompt for general conversational Q&A using FAISS memory.
    NOTE: For follow-up on last asset, we use a different path (Option A).
    """
    if retrieved_docs:
        context_texts = []
        for doc in retrieved_docs:
            meta_str = ", ".join(f"{k}={v}" for k, v in doc["metadata"].items())
            context_texts.append(f"- [score={doc['score']:.3f}, {meta_str}] {doc['text']}")
        context_block = "\n".join(context_texts)
    else:
        context_block = "No prior memory snippets were found relevant to this query."

    return dedent(f"""
        You are a helpful conversational financial assistant.

        You DO NOT have access to live market data in this mode.
        You MUST answer using ONLY the MEMORY SNIPPETS below and general financial knowledge.
        If the user asks for specific numbers like “current price” or “52-week high/low” and
        they are NOT present in the memory, say clearly that you don't have those numbers.

        =======================
        MEMORY SNIPPETS
        =======================
        {context_block}

        =======================
        USER MESSAGE
        =======================
        {user_message}

        =======================
        TASK
        =======================
        - Answer the user's question conversationally.
        - You may reference or summarize prior analyses in MEMORY SNIPPETS.
        - Do NOT fabricate exact prices, dates, or statistics not present in memory.
        - If you lack specific data, say so honestly.
    """)


# -----------------------------------------------------
# 6. FOLLOW-UP HELPERS (UNCHANGED LOGIC)
# -----------------------------------------------------
def answer_followup_stock(user_message: str, symbol: str) -> str | None:
    analysis = latest_equity_analysis.get(symbol)
    if not analysis:
        return None

    prompt = dedent(f"""
        You are answering a follow-up question about the stock {symbol}.

        Below is your PREVIOUS ANALYSIS for this stock.
        You must base your answer ONLY on this text.
        Do NOT bring in data about any other stocks unless they appear in the analysis.

        ======================
        PREVIOUS ANALYSIS
        ======================
        {analysis}

        ======================
        USER QUESTION
        ======================
        {user_message}

        ======================
        TASK
        ======================
        - Answer briefly and directly.
        - If the user asks for 52-week high/low, date range, sentiment, trend,
          or key risks, use the exact values/phrases from the analysis.
        - If the answer is not clearly contained in the analysis, say you don't have that information.
    """)
    return call_gemini(prompt)


def answer_followup_crypto(user_message: str, symbol: str) -> str | None:
    analysis = latest_crypto_analysis.get(symbol)
    if not analysis:
        return None

    prompt = dedent(f"""
        You are answering a follow-up question about the crypto asset {symbol}.

        Below is your PREVIOUS ANALYSIS for this asset.
        You must base your answer ONLY on this text.
        Do NOT bring in data about any other coins (e.g., ETH, SOL, DOGE)
        unless they appear in the analysis itself.

        ======================
        PREVIOUS ANALYSIS
        ======================
        {analysis}

        ======================
        USER QUESTION
        ======================
        {user_message}

        ======================
        TASK
        ======================
        - Answer briefly and directly.
        - If the user asks for 1-year high/low, 52-week high/low, date range,
          sentiment, or trend, use the exact values/phrases from the analysis.
        - If the answer is not clearly contained in the analysis, say you don't have that information.
        - Never mix this asset with any other asset.
    """)
    return call_gemini(prompt)


def answer_followup_portfolio(user_message: str) -> str | None:
    if not latest_portfolio_analysis:
        return None

    prompt = dedent(f"""
        You are answering a follow-up question about the user's portfolio.

        Below is your PREVIOUS PORTFOLIO ANALYSIS.
        You must base your answer ONLY on this text.

        ======================
        PREVIOUS ANALYSIS
        ======================
        {latest_portfolio_analysis}

        ======================
        USER QUESTION
        ======================
        {user_message}

        ======================
        TASK
        ======================
        - Answer briefly and directly.
        - If the user asks about risk level, diversification, or main drivers,
          use the wording from the analysis.
        - If the answer is not clearly contained in the analysis, say you don't have that information.
    """)
    return call_gemini(prompt)


# -----------------------------------------------------
# 6.1 FOLLOW-UP INTENT DETECTOR (UNCHANGED LOGIC)
# -----------------------------------------------------
def is_potential_followup(user_message: str, context: dict) -> bool:
    """
    Decide if a message is likely a follow-up to the last asset/portfolio.
    """
    if not context or context.get("type") is None:
        return False

    msg = user_message.strip()
    if not msg:
        return False

    lower = msg.lower()
    last_sym = context.get("symbol")

    symbol_candidates = re.findall(r"\b[A-Z]{2,6}(?:-USD)?\b", msg)
    if symbol_candidates:
        if last_sym:
            others = [s for s in symbol_candidates if s.upper() != last_sym.upper()]
            if others:
                return False
        else:
            return False

    generic_topics = [
        "liquidity",
        "diversification",
        "market efficiency",
        "efficient market",
        "beta",
        "alpha",
        "volatility",
        "risk premium",
        "capital structure",
        "time value of money",
        "explain",
        "what is",
        "why do",
        "why does",
        "how does",
        "difference between",
    ]
    if any(kw in lower for kw in generic_topics) and all(p not in lower for p in [" it", " this", " that"]):
        return False

    followup_keywords = [
        " it", " this", " that", " they",
        "current price", "price now", "today's price",
        "1-year high", "1 year high", "52-week", "52 week",
        "highest price", "lowest price",
        "sentiment", "trend", "market cap", "volatility",
        "return", "performance", "risk", "drawdown", "range",
    ]
    if any(kw in lower for kw in followup_keywords):
        return True

    if len(msg.split()) <= 7 and msg.endswith("?"):
        return True

    return False


# -----------------------------------------------------
# 7. HANDLER FUNCTIONS — WEB VERSIONS
# (Return strings instead of printing, logic preserved)
# -----------------------------------------------------
def handle_stock_command(user_input: str) -> str:
    """Handle 'stock SYMBOL' command and return analysis text."""
    global last_context

    parts = user_input.strip().split()
    if len(parts) != 2:
        return "❌ Invalid format. Use: stock SYMBOL (e.g., stock AAPL)"

    symbol = parts[1].upper()
    data = get_equity_data(symbol)
    prompt = build_equity_prompt(symbol, data)
    response = call_gemini(prompt)

    # Store full analysis in structured memory
    latest_equity_analysis[symbol] = response

    # Update last context
    last_context["type"] = "stock"
    last_context["symbol"] = symbol

    # Also store in FAISS memory
    memory.add(
        text=f"Equity analysis for {symbol}.\n\n{response}",
        metadata={
            "type": "stock",
            "symbol": symbol,
            "timestamp": dt.datetime.now(dt.timezone.utc).isoformat(),
        },
    )

    return response


def handle_crypto_command(user_input: str) -> str:
    """Handle 'crypto SYMBOL' command and return analysis text."""
    global last_context

    parts = user_input.strip().split()
    if len(parts) != 2:
        return "❌ Invalid format. Use: crypto SYMBOL (e.g., crypto BTC-USD)"

    symbol = parts[1].upper()
    data = get_crypto_data(symbol)
    prompt = build_crypto_prompt(symbol, data)
    response = call_gemini(prompt)

    latest_crypto_analysis[symbol] = response

    last_context["type"] = "crypto"
    last_context["symbol"] = symbol

    memory.add(
        text=f"Crypto analysis for {symbol}.\n\n{response}",
        metadata={
            "type": "crypto",
            "symbol": symbol,
            "timestamp": dt.datetime.now(dt.timezone.utc).isoformat(),
        },
    )

    return response


def handle_portfolio_from_text(holdings_text: str) -> str:
    """
    Web-friendly version of portfolio command.
    holdings_text: multiline string, each line SYMBOL/WEIGHT (e.g., AAPL/0.3)
    """
    global latest_portfolio_analysis, last_context

    lines = [ln.strip() for ln in holdings_text.splitlines() if ln.strip()]
    holdings = []
    for line in lines:
        if "/" not in line:
            return "Format error: each line must be SYMBOL/WEIGHT (e.g., AAPL/0.3)"
        sym, w_str = line.split("/", 1)
        try:
            w = float(w_str)
        except ValueError:
            return "Weight must be numeric (e.g., 0.25)"
        holdings.append({"symbol": sym.upper(), "weight": w})

    if not holdings:
        return "No holdings provided. Please enter at least one SYMBOL/WEIGHT line."

    try:
        stats = get_portfolio_history(holdings, period="1y")
    except ValueError as e:
        return f"Error computing portfolio: {e}"

    prompt = build_portfolio_prompt(stats, holdings)
    response = call_gemini(prompt)

    latest_portfolio_analysis = response
    last_context["type"] = "portfolio"
    last_context["symbol"] = None

    memory.add(
        text=f"Portfolio analysis.\nHoldings: {holdings}\n\n{response}",
        metadata={
            "type": "portfolio",
            "timestamp": dt.datetime.now(dt.timezone.utc).isoformat(),
        },
    )

    return response


def handle_chat_message(user_message: str) -> str:
    """
    General chat mode with follow-up detection.
    Returns assistant response as string.
    """
    t = last_context.get("type")
    sym = last_context.get("symbol")

    followup_response = None

    if is_potential_followup(user_message, last_context):
        if t == "stock" and sym:
            followup_response = answer_followup_stock(user_message, sym)
        elif t == "crypto" and sym:
            followup_response = answer_followup_crypto(user_message, sym)
        elif t == "portfolio":
            followup_response = answer_followup_portfolio(user_message)

    if followup_response:
        response = followup_response
    else:
        retrieved = memory.search(user_message, k=5)
        prompt = build_chat_prompt(user_message, retrieved)
        response = call_gemini(prompt)

    memory.add(
        text=f"User: {user_message}\nAssistant: {response}",
        metadata={
            "type": "chat",
            "timestamp": dt.datetime.now(dt.timezone.utc).isoformat(),
        },
    )

    return response


# -----------------------------------------------------
# 8. SINGLE-STEP AGENT DISPATCH (replaces CLI loop)
# -----------------------------------------------------
def agent_step(user_input: str, portfolio_text: str | None = None) -> str:
    """
    Single step of the finance agent:
    - Detect command type
    - Route to appropriate handler
    - Return response text
    """
    if not user_input.strip():
        return "⚠️ Please enter a command or question."

    lower = user_input.lower().strip()

    if lower in ("quit", "exit", "q"):
        return "Session end requested. (In web mode, just stop sending queries.)"
    elif lower.startswith("stock "):
        return handle_stock_command(user_input)
    elif lower.startswith("crypto "):
        return handle_crypto_command(user_input)
    elif lower.startswith("portfolio"):
        if not portfolio_text or not portfolio_text.strip():
            return "Please provide holdings in the Portfolio Holdings box (one SYMBOL/WEIGHT per line)."
        return handle_portfolio_from_text(portfolio_text)
    else:
        return handle_chat_message(user_input)


# -----------------------------------------------------
# 9. FLASK APP & ROUTE
# -----------------------------------------------------
app = Flask(__name__)


@app.route("/", methods=["GET", "POST"])
def home():
    response = ""
    error = ""
    user_input = ""
    portfolio_text = ""

    if request.method == "POST":
        user_input = request.form.get("user_input", "").strip()
        portfolio_text = request.form.get("portfolio_text", "").strip()

        try:
            response = agent_step(user_input, portfolio_text)
        except Exception as e:
            error = f"⚠️ Unexpected error: {e}"

    return render_template(
        "index.html",
        response=response,
        error=error,
        user_input=user_input,
        portfolio_text=portfolio_text,
    )


# -----------------------------------------------------
# 10. RUN FLASK APP
# -----------------------------------------------------
if __name__ == "__main__":
    # In Colab this will be proxied via ngrok.
    app.run(host="0.0.0.0", port=8000, debug=False)

**Create folders**

In [ ]:
!mkdir -p templates
!mkdir -p static

**templates/index.html**

In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <title>💹 Finance Agent with Memory</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}" />
</head>
<body>
    <div class="hero-section">
        <div class="overlay"></div>

        <div class="hero-content">
            <h1>💹 Finance Agent with Memory</h1>
            <p>
                Analyze stocks, crypto, and portfolios using live Yahoo Finance data, Gemini reasoning,
                and semantic memory (FAISS).
            </p>

            <div class="help-box">
                <p><strong>Commands:</strong></p>
                <ul>
                    <li><code>stock AAPL</code> → Equity analysis for AAPL</li>
                    <li><code>crypto BTC-USD</code> → Crypto analysis for BTC-USD</li>
                    <li><code>portfolio</code> → Analyze portfolio (use holdings box)</li>
                    <li><em>Anything else</em> → Chat using memory & follow-ups</li>
                </ul>
            </div>

            <form method="post" class="card">
                <label for="user_input" class="field-label">Your command or question</label>
                <textarea
                    id="user_input"
                    name="user_input"
                    rows="3"
                    placeholder="Example: stock AAPL, crypto BTC-USD, or 'Explain the main risk in my last portfolio analysis.'"
                    required
                >{{ user_input }}</textarea>

                <label for="portfolio_text" class="field-label">
                    Portfolio holdings (only used when command is <code>portfolio</code>)
                </label>
                <textarea
                    id="portfolio_text"
                    name="portfolio_text"
                    rows="4"
                    placeholder="One per line: SYMBOL/WEIGHT (e.g., AAPL/0.4&#10;MSFT/0.3&#10;GOOGL/0.3)"
                >{{ portfolio_text }}</textarea>

                {% if error %}
                <p class="error-msg">{{ error }}</p>
                {% endif %}

                <button type="submit" class="btn-primary">Run Finance Agent 🚀</button>
            </form>
        </div>
    </div>

    {% if response %}
    <div class="result-section fade-in">
        <div class="result-card">
            <h2>📊 Agent Response</h2>
            <p class="result-text">{{ response }}</p>
        </div>
    </div>
    {% endif %}
</body>
</html>

**static/style.css**

In [ ]:
%%writefile static/style.css
@import url('https://fonts.googleapis.com/css2?family=Poppins:wght@400;600&display=swap');

body {
    margin: 0;
    font-family: 'Poppins', sans-serif;
    color: #fff;
}

/* Hero background */
.hero-section {
    position: relative;
    min-height: 100vh;
    display: flex;
    align-items: center;
    padding: 40px 8%;
    background: linear-gradient(135deg, #10131a, #1f3147);
}

.overlay {
    position: absolute;
    inset: 0;
    background: rgba(0,0,0,0.35);
}

/* Main content card */
.hero-content {
    position: relative;
    z-index: 2;
    max-width: 780px;
    background: rgba(255,255,255,0.08);
    border-radius: 24px;
    padding: 28px 32px;
    backdrop-filter: blur(8px);
    box-shadow: 0 10px 30px rgba(0,0,0,0.45);
}

h1 {
    margin: 0 0 10px;
    font-size: 2rem;
    color: #ffdf9e;
}

p {
    margin: 4px 0;
}

/* Helper box */
.help-box {
    margin-top: 12px;
    padding: 10px 12px;
    background: rgba(0,0,0,0.35);
    border-radius: 12px;
    font-size: 0.9rem;
}

.help-box code {
    background: rgba(255,255,255,0.1);
    padding: 1px 4px;
    border-radius: 4px;
}

/* Form card */
.card {
    margin-top: 18px;
    padding: 16px 18px;
    background: rgba(0,0,0,0.35);
    border-radius: 16px;
    display: flex;
    flex-direction: column;
    gap: 12px;
}

.field-label {
    font-size: 0.9rem;
}

textarea {
    width: 100%;
    padding: 10px;
    border-radius: 10px;
    border: none;
    resize: vertical;
    background: rgba(255,255,255,0.95);
    color: #111;
    font-size: 0.95rem;
}

/* Button */
.btn-primary {
    padding: 10px 12px;
    border: none;
    border-radius: 10px;
    background: linear-gradient(135deg, #00c6ff, #0072ff);
    color: #fff;
    font-weight: 600;
    cursor: pointer;
    font-size: 0.95rem;
    align-self: flex-start;
    transition: 0.25s ease;
}

.btn-primary:hover {
    transform: translateY(-2px);
    box-shadow: 0 6px 16px rgba(0,0,0,0.35);
}

/* Error */
.error-msg {
    margin-top: 4px;
    color: #ffb3b3;
    font-size: 0.9rem;
}

/* Result section */
.result-section {
    background: #02040a;
    padding: 40px 20px;
    display: flex;
    justify-content: center;
}

.result-card {
    max-width: 900px;
    width: 100%;
    background: rgba(255,255,255,0.05);
    padding: 24px 28px;
    border-radius: 16px;
    color: #e2e5ff;
}

.result-card h2 {
    margin-top: 0;
}

.result-text {
    white-space: pre-wrap;
    line-height: 1.5;
}

/* Animation */
.fade-in {
    animation: fadeInUp 0.6s ease forwards;
}

@keyframes fadeInUp {
    from { opacity: 0; transform: translateY(18px); }
    to   { opacity: 1; transform: translateY(0); }
}

**Start Flask + ngrok**

In [ ]:
# Kill any running Flask/ngrok processes
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"

In [ ]:
!lsof -i :8000

In [ ]:
!kill -9 617

In [ ]:
# Start Flask in background
!nohup python app.py > flask.log 2>&1 &

In [ ]:
# Start ngrok tunnel
from pyngrok import ngrok, conf

# Enter your NGROK auth token here
conf.get_default().auth_token = "INPUT_YOUR_NGROK_TOKEN_HERE"  # replace if needed

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)